In [1]:
import sys
sys.path

['',
 '//anaconda/envs/py27/lib/python2.7/site-packages/twitter-1.17.1-py2.7.egg',
 '//anaconda/envs/py27/lib/python27.zip',
 '//anaconda/envs/py27/lib/python2.7',
 '//anaconda/envs/py27/lib/python2.7/plat-darwin',
 '//anaconda/envs/py27/lib/python2.7/plat-mac',
 '//anaconda/envs/py27/lib/python2.7/plat-mac/lib-scriptpackages',
 '//anaconda/envs/py27/lib/python2.7/lib-tk',
 '//anaconda/envs/py27/lib/python2.7/lib-old',
 '//anaconda/envs/py27/lib/python2.7/lib-dynload',
 '/anaconda/envs/py27/lib/python2.7/site-packages/setuptools-19.1.1-py2.7.egg',
 '//anaconda/envs/py27/lib/python2.7/site-packages',
 '//anaconda/envs/py27/lib/python2.7/site-packages/IPython/extensions',
 '/Users/linwoodc3/.ipython']

In [ ]:
# Import the necessary package to process data in JSON format
try:
    import json
except ImportError:
    import simplejson as json

# Import the necessary methods from "twitter" library
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

# Variables that contains the user credentials to access Twitter API 
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)

# Get a sample of the public data following through Twitter
iterator = twitter_stream.statuses.filter(locations = '-76.871724,38.902399, -76.859321,38.911399')

# Print each tweet in the stream to the screen 
# Here we set it to stop after getting 1000 tweets. 
# You don't have to set it to stop, but can continue running 
# the Twitter API to collect data for days or even longer. 

with open('data.txt', 'w+') as outfile:
    for tweet in iterator:
        
        # Twitter Python Tool wraps the data returned by Twitter 
        # as a TwitterDictResponse object.
        # We convert it back to the JSON format to print/score
        #print json.dumps(tweet)  

        # The command below will do pretty printing for JSON data, try it out
        # print json.dumps(tweet, indent=4)
        json.dump(tweet, outfile)
    outfile.closed

In [4]:
file = open('/Users/linwoodc3/projects/LC3-Creations/notebooks/twittertest/data.txt', 'r')
#b = json.loads("")

In [5]:
file

<open file '/Users/linwoodc3/projects/LC3-Creations/notebooks/twittertest/data.txt', mode 'r' at 0x10370a930>

In [7]:
import sys
sys.getsizeof(file)

144

In [12]:
print file.readline(3000)

, "indices": [43, 66], "type": "photo", "id": 686299088757731328, "media_url": "http://pbs.twimg.com/media/CYY5YpMUMAAEuxk.jpg"}]}, "in_reply_to_screen_name": null, "id_str": "686299098371256320", "retweet_count": 0, "in_reply_to_user_id": null, "favorited": false, "user": {"follow_request_sent": null, "profile_use_background_image": false, "default_profile_image": false, "id": 78427473, "verified": false, "profile_image_url_https": "https://pbs.twimg.com/profile_images/684746209454592000/nXcvpy5t_normal.jpg", "profile_sidebar_fill_color": "000000", "profile_text_color": "000000", "followers_count": 12734, "profile_sidebar_border_color": "000000", "id_str": "78427473", "profile_background_color": "000000", "listed_count": 353, "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "utc_offset": -18000, "statuses_count": 45390, "description": "I work in radio & TV for @Fanduel, @masnsports, @scoutfantasy, @SiriusXM, @wusa9, @1067thefandc Daddy, husban

In [1]:
import json
import re

#shameless copy paste from json/decoder.py
FLAGS = re.VERBOSE | re.MULTILINE | re.DOTALL
WHITESPACE = re.compile(r'[ \t\n\r]*', FLAGS)

class ConcatJSONDecoder(json.JSONDecoder):
    def decode(self, s, _w=WHITESPACE.match):
        s_len = len(s)

        objs = []
        end = 0
        while end != s_len:
            obj, end = self.raw_decode(s, idx=_w(s, end).end())
            end = _w(s, end).end()
            objs.append(obj)
        return objs

In [2]:
with open('/Users/linwoodc3/projects/LC3-Creations/notebooks/twittertest/data.txt', 'r') as myfile:
    data=myfile.read()

In [3]:
ConcatJSONDecoder

__main__.ConcatJSONDecoder

In [4]:
b = json.load(open('/Users/linwoodc3/projects/LC3-Creations/notebooks/twittertest/data.txt'), cls=ConcatJSONDecoder)

In [49]:
b[3]['user']['location']

u'in the 6IX with Aubrey '

In [50]:
for l in b[1:50]:
    print l['user']['location']

Chesterfield, Virginia
Brasil / Mexico
in the 6IX with Aubrey 
None
Washington, D.C.
Timberville, VA
Virginia.
None
maryland
Virginia, USA
PHS |⚡️804⚡️ | RVA 
Virginia, USA
None
Haymarket, VA
Fairfax, VA
Richmond, VA
804//DRHS
USA
Va
None
The 410
None
Virginia, USA
None
Triangle, VA
Church Hill, TN but ❤️ in LA
None
757 // PG County, 〽️urrland
Snapchat: Chucktaylor6
Atlee High School
The Trenches
Yorktown, VA
Pittsburgh ➡ Annapolis⚓⛵
301
Washington, DC
Morgantown, WV
Washington, DC
None
Baltimore, MD
Washington, DC
Washington, DC
None
Richmond, VA
Washington, DC
Baltimore, MD
Virginia, USA
None
None
Washington, DC


In [52]:
skins_vs_pack_locs = [l['user']['location'] for l in b]
    

In [53]:
len(skins_vs_pack_locs)

8452

In [ ]:
for l in set(skins_vs_pack_locs):
    try:
        print geolocator.geocode(l)
    except:
        pass

In [ ]:
skins_vs_pack_locs[1:100]

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()


In [10]:
geolocator.geocode('Richmond, Virginia')

GeocoderServiceError: <urlopen error [Errno 61] Connection refused>

In [16]:
len(geolocator.geocode('College Park, MD'))

2

In [17]:
skins_vs_pack_truelocs = []
for l['user']['location'] in b:
    try:
        if len(geolocator.geocode(l['user']['location'])) > 0:
            skins_vs_pack_truelocs.append(l['user']['location'])
    except:
        pass   

In [18]:
len(skins_vs_pack_truelocs)

0

In [20]:
for l in b[1:10]:
    print l['user']['location']

Chesterfield, Virginia
Brasil / Mexico
{u'contributors': None, u'truncated': False, u'text': u'My baby daddy https://t.co/4DD3Gc2i3M', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 686299562491953152, u'favorite_count': 0, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1452461541184', u'entities': {u'user_mentions': [], u'symbols': [], u'hashtags': [], u'urls': [], u'media': [{u'expanded_url': u'http://twitter.com/mrspoohbear20/status/686299562491953152/photo/1', u'sizes': {u'large': {u'h': 656, u'w': 620, u'resize': u'fit'}, u'small': {u'h': 359, u'w': 340, u'resize': u'fit'}, u'medium': {u'h': 634, u'w': 600, u'resize': u'fit'}, u'thumb': {u'h': 150, u'w': 150, u'resize': u'crop'}}, u'url': u'https://t.co/4DD3Gc2i3M', u'media_url_https': u'https://pbs.twimg.com/media/CYY5zqQUQAAgEOO.jpg', u'id_str': u'686299552899416064', u'indices': [14, 37], u'media_u